# Quora Question Pairs Competition Notebook

### To Improve User (question seeker, answer writer) Satisfaction by buidling a model to improve model performance of low log loss.

In [10]:
# Import necessary libraries
import pandas as pd
import nltk
import re
import utilities as u
import numpy as np
import sklearn
import pickle
import collections
import keras
import tensorflow as tf;
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix
from keras.layers import Input, LSTM, Dense
from keras.models import Model
from keras.layers import Input, Embedding, LSTM
from keras.preprocessing.text import Tokenizer
from gensim.models import KeyedVectors


#Check Keras and tensorflow version
print ("Importing modules and Loading word2vec pretrained embedding vector")
print("keras version is {}".format(keras.__version__))
print("tf version is {}".format(tf.__version__))


Importing modules and Loading word2vec pretrained embedding vector
keras version is 2.1.2
tf version is 1.4.1


In [5]:
# Load pretrained embedding vector trained on Google News
EMBEDDING_FILE = '../GoogleNews-vectors-negative300.bin.gz'
model = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary=True)

# To check if word2vec embedding model is loaded succesfully
result = model.most_similar(positive=['woman', 'king'], negative=['man'], topn=1)
print(result)

/Users/jac812i/virtualenvs/quoravenv/lib/python3.6/site-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


[('queen', 0.7118192911148071)]


In [11]:
# Read training and test data from .csv files
train_df = pd.read_csv("../train.csv")
test_df = pd.read_csv("../test.csv")

/Users/jac812i/virtualenvs/quoravenv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [84]:
# Display top 3 rows of train data
train_df.head(3)
# assert the shape of train dataframe
assert train_df.shape[0] > 0
assert train_df.shape[1] > 0

In [85]:
# Display top 3 rows of test data and assert test data shape
test_df.head(3)
assert test_df.shape[0] > 0
assert test_df.shape[1] > 0

# Identify Feature and Target columns

In [14]:
# Extract feature (X) and target (y) columns
feature_cols = list(train_df.columns[:-1])
target_col = train_df.columns[-1]
print ("Feature Columns {}".format(feature_cols))
print ("Target Columns {}".format(target_col))
X_all = train_df[feature_cols]
y_all = pd.DataFrame(data=train_df[target_col], columns=[target_col])

Feature Columns ['id', 'qid1', 'qid2', 'question1', 'question2']
Target Columns is_duplicate


In [15]:
# To check if the data is balanced or not
collections.Counter(y_all['is_duplicate'])

Counter({0: 255027, 1: 149263})

In [16]:
# Check the 5 rows of train output
y_all.head(5)

,is_duplicate
0,0
1,0
2,0
3,0
4,0


In [22]:
# Check the 3 rows of the new dataframe
X_all.head(3)
# Assert the shapes are equal after removing the feature
assert X_all.shape == (train_df.shape[0],train_df.shape[1]-1)

# Split data in training and validation sets

In [61]:
# Split data into 70% training and 30% validation and have a constant random state so that the results are consistent
X_train, X_valid, y_train, y_valid = train_test_split(X_all, y_all, test_size=0.30, random_state=42)
print ("Total data set: {} samples".format(X_all.shape[0]))
print ("Training set: {} samples".format(X_train.shape[0]))
print ("Valid set: {} samples".format(X_valid.shape[0]))

Total data set: 404290 samples
Training set: 283003 samples
Valid set: 121287 samples


In [62]:
# asserting the shapes after splitting the data
assert (X_train.shape[0] + X_valid.shape[0]) == X_all.shape[0]
assert (y_train.shape[0] + y_valid.shape[0]) == y_all.shape[0]

# Check whether data is balanced or not

In [63]:
# TO check how many question pairs in training set are duplicate or not
collections.Counter(y_train['is_duplicate'])

Counter({0: 178677, 1: 104326})

In [64]:
# TO check how many question pairs in validation set are duplicate or not
collections.Counter(y_valid['is_duplicate'])

Counter({0: 76350, 1: 44937})

In [65]:

#Check few rows of X_valid
X_valid.head(3)

,id,qid1,qid2,question1,question2
8067,8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?
368101,368101,12736,104117,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...
70497,70497,121486,121487,Which is more advisable and better material fo...,What is the best server setup for buddypress?


In [66]:
#Check few rows of y_valid
y_valid.head(3)

,is_duplicate
8067,0
368101,0
70497,0


In [67]:
# Reset index to start from 0
X_train.reset_index(drop = True, inplace = True)
X_valid.reset_index(drop = True, inplace = True)
y_train.reset_index(drop = True, inplace = True)
y_valid.reset_index(drop = True, inplace = True)

#asserting shapes after resetting index
assert (X_train.shape[0] + X_valid.shape[0]) == X_all.shape[0]
assert (y_train.shape[0] + y_valid.shape[0]) == y_all.shape[0]

In [68]:

#Check few rows of X_valid
X_valid.head(3)

,id,qid1,qid2,question1,question2
0,8067,15738,15739,How do I play Pokémon GO in Korea?,How do I play Pokémon GO in China?
1,368101,12736,104117,What are some of the best side dishes for crab...,What are some good side dishes for buffalo chi...
2,70497,121486,121487,Which is more advisable and better material fo...,What is the best server setup for buddypress?


### Simple Data Preprocessing for Baseline

In [69]:
# Function that takes a text and preprocess it like converting to string, split the string etc

def preprocess_baseline_text(text):
    text = str(text)
    text = text.lower()
    text = text.split()
    
    return text


#Test the function
assert preprocess_baseline_text('How do I play Pokémon GO in Korea?') == ['how', 'do', 'i', 'play', 'pokémon', 'go', 'in', 'korea?']


In [77]:
print (X_valid[0:1])

     id   qid1   qid2                           question1  \
0  8067  15738  15739  How do I play Pokémon GO in Korea?   

                            question2  
0  How do I play Pokémon GO in China?  


In [83]:
# Function that returns a list of all the words from a dataframe

def create_vocabulary(words_list,df,column_name):
    for sentence in df[column_name]:
        for word in sentence:
            words_list.append(word)
    return words_list


#Test the function
data = [['How do I play Pokémon GO in Korea?']]
test_sub_df = pd.DataFrame(data,columns=['question1'])
test_sub_df['question1'] = test_sub_df['question1'].apply(lambda x: preprocess_baseline_text(x))

# assert the function is working fine with dataframe and column name
assert create_vocabulary([],test_sub_df,'question1') == ['how', 'do', 'i', 'play', 'pokémon', 'go', 'in', 'korea?']


In [104]:
# Create new dataframes and do data preprocessing on the question1 and question 2 columns
X_train_df = pd.DataFrame()
X_valid_df = pd.DataFrame()
X_train_df['question1'] = X_train['question1'].apply(lambda x:preprocess_baseline_text(x))
X_train_df['question2'] = X_train['question2'].apply(lambda x:preprocess_baseline_text(x))
X_valid_df['question1'] = X_valid['question1'].apply(lambda x:preprocess_baseline_text(x))
X_valid_df['question2'] = X_valid['question2'].apply(lambda x:preprocess_baseline_text(x))

#assertions on dataframe shape's
assert X_train_df.shape[0] == X_train.shape[0]
assert X_train_df.shape[1] == 2
assert X_valid_df.shape[0] == X_valid_df.shape[0]
assert X_valid_df.shape[1] == 2

In [105]:
# Check few rows of dataframe for manual validation
X_train_df.head(3)

,question1,question2
0,"[how, is, the, working, environment, at, sbi, ...","[how, stressful, is, work, of, sbi, clerk?]"
1,"[how, can, a, us, citizen, work, in, canada?]","[will, a, us, graduate, degree, help, a, non-u..."
2,"[what, are, the, benefits, of, washing, your, ...","[why, is, it, important, to, wash, your, hands..."


In [106]:
# Check few rows of dataframe for manual validation
X_valid_df.head(3)

,question1,question2
0,"[how, do, i, play, pokémon, go, in, korea?]","[how, do, i, play, pokémon, go, in, china?]"
1,"[what, are, some, of, the, best, side, dishes,...","[what, are, some, good, side, dishes, for, buf..."
2,"[which, is, more, advisable, and, better, mate...","[what, is, the, best, server, setup, for, budd..."


In [96]:
# Create complete vocabulary from all the data using training and test data.

words_list = create_vocabulary([],X_all_1,'question1')
print ("Length of words after adding training data question 1 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,X_all_1,'question2')
print ("Length of words after adding training data question 2 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,test_df_1,'question1')
print ("Length of words after adding test data question 1 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,test_df_1,'question2')
print ("Length of words after adding test data question 2 {}".format(len(words_list)))

# asserting to check words list calculation did not provide empty records
assert len(words_list) > 0


Length of words after adding training data question 1 4423826
Length of words after adding training data question 2 8944593
Length of words after adding test data question 1 48146246
Length of words after adding test data question 2 87470531


# Create a simple baseline model with Logistic Regression
###    Create a feature based on how many words are common in question 1 and question 2

In [107]:
# Function to calculate how many unique words are common between two question sentence's
def termfrequency(sentence1, sentence2):
   
    question_dict ={}
    sentence1_words = sentence1   
    sentence2_words = sentence2
    searchtermfreq = []
    
    for key in sentence1_words:
        question_dict[key] = question_dict.get(key,0) + 1
    
    for key in set(sentence2_words):
        value =  question_dict.get(key,0)
        if value >= 1:
            value = 1
        searchtermfreq.append(value)
        
    return sum(searchtermfreq)


#assertion to make sure term frequency function is working fine
assert termfrequency(['what', 'is', 'the', 'step', 'by', 'step'], ['what', 'is', 'the', 'step', 'by', 'step', 'guide']) == 5

In [130]:
#Creating a new column common_term_freq which has the value of unique words that are common between two question sentence's

X_train_df['common_term_freq'] = X_train_df.apply(lambda x: termfrequency(x['question1'],x['question2']), axis=1 )
X_valid_df['common_term_freq'] = X_valid_df.apply(lambda x: termfrequency(x['question1'],x['question2']), axis=1 )

# assert to make sure the column is a number
assert X_train_df['common_term_freq'].shape[0] == X_train_df.shape[0]
assert X_valid_df['common_term_freq'].shape[0] == X_valid_df.shape[0]
assert X_train_df['common_term_freq'][0] >- 0
assert X_valid_df['common_term_freq'][0] >- 0

### Creating features based on Total words frequency

In [131]:
#Function to calculate count of number of words in a sentence
def total_words_freq(sentence):
    return len(sentence)

assert total_words_freq(['what', 'is', 'the', 'step', 'by', 'step']) == 6

In [133]:
#Create two new column's question1_words_freq and question2_words_freq using the above function 
#to calculate count of number of words in a sentence 

X_train_df['question1_words_freq'] = X_train_df['question1'].map(lambda x: total_words_freq(x))
X_train_df['question2_words_freq'] = X_train_df['question2'].map(lambda x: total_words_freq(x))

X_valid_df['question1_words_freq'] = X_valid_df['question1'].map(lambda x: total_words_freq(x))
X_valid_df['question2_words_freq'] = X_valid_df['question2'].map(lambda x: total_words_freq(x))


# assert to make sure the column is a number
assert X_train_df['question1_words_freq'][0] >= 0
assert X_train_df['question2_words_freq'][0] >= 0
assert X_valid_df['question1_words_freq'][0] >= 0
assert X_valid_df['question2_words_freq'][0] >= 0



In [134]:
# Check few rows
X_train_df.head(3)


,question1,question2,common_term_freq,question1_words_freq,question2_words_freq
0,"[how, is, the, working, environment, at, sbi, ...","[how, stressful, is, work, of, sbi, clerk?]",3,9,7
1,"[how, can, a, us, citizen, work, in, canada?]","[will, a, us, graduate, degree, help, a, non-u...",7,8,20
2,"[what, are, the, benefits, of, washing, your, ...","[why, is, it, important, to, wash, your, hands...",4,10,10


In [135]:
# Check few rows
X_valid_df.head(3)

,question1,question2,common_term_freq,question1_words_freq,question2_words_freq
0,"[how, do, i, play, pokémon, go, in, korea?]","[how, do, i, play, pokémon, go, in, china?]",7,8,8
1,"[what, are, some, of, the, best, side, dishes,...","[what, are, some, good, side, dishes, for, buf...",6,11,9
2,"[which, is, more, advisable, and, better, mate...","[what, is, the, best, server, setup, for, budd...",2,16,8


In [139]:
# Drop column's question1 and question2 since machine learning input requires columns with numbers
X_train_model_input = X_train_df.drop(['question1','question2'],axis =1)
X_valid_model_input = X_valid_df.drop(['question1','question2'],axis =1)

assert X_train_model_input.shape[1] == X_train_df.shape[1] - 2
assert X_valid_model_input.shape[1] == X_valid_df.shape[1] - 2

# Baseline Model Training and Evaluation

In [ ]:


def train_classifier(clf, X_train, y_train):
    print ("Training {}...".format(clf.__class__.__name__))
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print ("Done!\nTraining time (secs): {:.3f}".format(end - start))
    



clf =  LogisticRegression()

train_classifier(clf, X_train_model_input, y_train.values.ravel())



def predict_labels(clf, X_train, y_train):
    print ("Predicting labels using {}...".format(clf.__class__.__name__))
    start = time.time()
    y_pred = clf.predict(X_train)
    end = time.time()
    print ("Done!\nPrediction time (secs): {:.3f}".format(end - start))
    return log_loss(y_train, y_pred, eps=1e-15), confusion_matrix(y_train, y_pred)

train_metrics = predict_labels(clf, X_train_model_input, y_train.values.ravel())

print 
print ("Log loss for training set: {}".format(train_metrics[0]))

print ("Confusion matrix for training set: {}".format(train_metrics[1]))

# Predict on test data
print ("Log loss for validation set: {}".format(predict_labels(clf, X_valid_model_input, y_valid.values.ravel())[0]))

print ("Confusion matrix for validation set: {}".format(predict_labels(clf, X_valid_model_input, y_valid.values.ravel())[1]))




# To check how many top frequent 10000 words in complete vocabulary are in word2vec 

In [ ]:
# To check how many top frequent 10000 words in complete vocabulary are in word2vec after basic preprocessing of text

print ("Total count of unique words in vocabulary {}".format(len(set(words_list)))
# print (words_list[0:10])
words_freq = collections.Counter(words_list)
words_freq.most_common(10)
words_freq_10000 = words_freq.most_common(10000)


word_in_word2vec = []
word_notin_word2vec = []

for word in words_freq.most_common(10000):
    if word[0] in model.vocab:
        word_in_word2vec.append(word[0])
    else:
        word_notin_word2vec.append(word[0])
        
print ("Out of 10000 top frequent words {} words are present in word2vec after basic preprocessing of text".format(len(word_in_word2vec)))
print ("Out of 10000 top frequent words {} words are not present in word2vec after basic preprocessing of text".format(len(word_notin_word2vec)))

     
        

In [ ]:


print ("Top 100 words from hightly frequent words in vocabulary present in word2vec {}".format(words_freq_10000[0:100]))

In [ ]:
print ("Top 100 words from hightly frequent words in vocabulary not present in word2vec {} {}".format(word_notin_word2vec[0:100]))


In [ ]:
print ("List the first 100 words present that are present in vocabulary and in word2vec")
print (word_in_word2vec[0:100])

# Further Preprocessing Text

In [ ]:
# As you can see from above there are many words that are not present in word2vec and some of them are 

In [ ]:
def preprocess_text(text):
    text = str(text)
    text = text.lower()
    
    text = re.sub(r"[^A-Za-z0-9^?,!.\/'+-=]", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\?", " ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"!", " ! ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\+", " + ", text)
    text = re.sub(r"\-", " - ", text)
    text = re.sub(r"\=", " = ", text)
    
    text = text.split()
    
    return text

In [ ]:


X_train_df = pd.DataFrame()
X_valid_df = pd.DataFrame()
X_test_df = pd.DataFrame()

In [ ]:
X_train.head(3)

In [ ]:
test_df.head(3)

In [ ]:
preprocess_text('What is= the step by step guide to invest in?')

In [ ]:
X_train_df['question1'] = X_train['question1'].apply(lambda x:preprocess_text(x))
X_train_df['question2'] = X_train['question2'].apply(lambda x:preprocess_text(x))
X_valid_df['question1'] = X_valid['question1'].apply(lambda x:preprocess_text(x))
X_valid_df['question2'] = X_valid['question2'].apply(lambda x:preprocess_text(x))
X_test_df['question1'] = test_df['question1'].apply(lambda x:preprocess_text(x))
X_test_df['question2'] = test_df['question2'].apply(lambda x:preprocess_text(x))

In [ ]:
X_train_df.head(3)

In [ ]:
X_train_df['question1'].shape

In [ ]:
words_list = create_vocabulary([],X_train_df,'question1')
print ("Lenght of words in X_train_df question 1 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,X_train_df,'question2')
print ("Lenght of words after adding X_train_df question 2 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,X_valid_df,'question1')
print ("Lenght of words after adding X_valid_df question 1 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,X_valid_df,'question2')
print ("Lenght of words after adding X_valid_df question 2 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,X_test_df,'question1')
print ("Lenght of words after adding X_test_df question 1 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,X_test_df,'question2')
print ("Lenght of words after adding X_test_df question 2 {}".format(len(words_list)))

In [ ]:
print (len(set(words_list)))
print (words_list[0:10])
words_freq = collections.Counter(words_list)
words_freq.most_common(10)
words_freq_10000 = words_freq.most_common(10000)


word_in_word2vec = []
word_notin_word2vec = []

for word in words_freq.most_common(10000):
    if word[0] in model.vocab:
        word_in_word2vec.append(word[0])
    else:
        word_notin_word2vec.append(word[0])
        
print (len(word_in_word2vec))
print (len(word_notin_word2vec))

In [ ]:
print (word_notin_word2vec[0:100]) 

# Tokenizer

# Keras Implementation

In [ ]:
words_to_index = {}
index_to_words = {}

i = 0
for word in set(words_list):
    words_to_index[word] = i
    i = i + 1
    
j=0
for word in set(words_list):
    index_to_words[j] = word
    j=j+1

In [ ]:
import pickle
with open('words_to_index_v2.pickle', 'wb') as handle:
    pickle.dump(words_to_index, handle, protocol=pickle.HIGHEST_PROTOCOL)

with open('index_to_words_v2.pickle', 'wb') as handle:
    pickle.dump(index_to_words, handle, protocol=pickle.HIGHEST_PROTOCOL)    
    
    



In [ ]:
print (words_to_index['the'])
print (len(words_to_index))
print (index_to_words[104295])
print (index_to_words[0])

In [ ]:
def word_2_integer(wordslist):
    question2integer = []
    for word in wordslist:
        question2integer.append(words_to_index[word])

    return question2integer

word_2_integer(['what', 'is', 'the', 'story', 'of', 'kohinoor'])

In [ ]:
X_train_df['question1'] = X_train_df['question1'].apply(lambda x:word_2_integer(x))


In [ ]:
X_train_df['question2'] = X_train_df['question2'].apply(lambda x:word_2_integer(x))
X_valid_df['question1'] = X_valid_df['question1'].apply(lambda x:word_2_integer(x))
X_valid_df['question2'] = X_valid_df['question2'].apply(lambda x:word_2_integer(x))
X_test_df['question1'] = X_test_df['question1'].apply(lambda x:word_2_integer(x))
X_test_df['question2'] = X_test_df['question2'].apply(lambda x:word_2_integer(x))

In [ ]:
X_train_df['question1'].head(5)

In [ ]:
import numpy as np
embed_length = 300
embed_matrix = np.random.randn(len(words_to_index)+1,embed_length)
# To ignore padding
embed_matrix[0] = 0



In [ ]:
print (embed_matrix.shape)

In [ ]:
print (embed_matrix[1])

In [ ]:
#Updating embedding matrix 
count = 0
for word, index in words_to_index.items():
    if word in model.vocab:
        count = count + 1
        embed_matrix[index] = model.word_vec(word)

print (count)


In [ ]:
print (len(words_to_index))

In [ ]:
max_seq_length = max([
max(list(X_train_df['question1'].map(lambda x: len(x)))),
max(list(X_train_df['question2'].map(lambda x: len(x)))),
max(list(X_valid_df['question1'].map(lambda x: len(x)))),
max(list(X_valid_df['question2'].map(lambda x: len(x)))),
max(list(X_test_df['question1'].map(lambda x: len(x)))),
max(list(X_test_df['question2'].map(lambda x: len(x)))),
])

In [ ]:
print (max_seq_length)

In [ ]:
# Convert labels to their numpy representations
Y_train =  y_train.values
Y_valid =  y_valid.values


In [ ]:
print (type(Y_train))
print (type(Y_valid))

In [ ]:
print (Y_train[0:10])

In [ ]:
from tensorflow.python.keras import utils

In [ ]:
Y_train = utils.to_categorical(Y_train, num_classes=2)
Y_valid = utils.to_categorical(Y_valid, num_classes=2)

In [ ]:
# assert X_train_dict['left'].shape == X_train_dict['right'].shape

In [ ]:
# assert len(X_train_dict['left']) == len(Y_train)


In [ ]:
#padding to max seq length

In [ ]:
X_train_df['question1'][0]

In [ ]:
def pad_left_zeros(question_list,max_seq_length):
    question_list = [0] * (max_seq_length - len(question_list)) + question_list
    return question_list

In [ ]:
# len (pad_left_zeros([31586, 69984, 104295, 57112, 55384, 21628, 63296, 131479, 95639]))

In [ ]:
X_train_df['question1'] = X_train_df['question1'].apply(lambda x: pad_left_zeros(x,max_seq_length))

In [ ]:
X_train_df['question2'] = X_train_df['question2'].apply(lambda x: pad_left_zeros(x,max_seq_length))
X_valid_df['question1'] = X_valid_df['question1'].apply(lambda x: pad_left_zeros(x,max_seq_length))
X_valid_df['question2'] = X_valid_df['question2'].apply(lambda x: pad_left_zeros(x,max_seq_length))
X_test_df['question1'] = X_test_df['question1'].apply(lambda x: pad_left_zeros(x,max_seq_length))
X_test_df['question2'] = X_test_df['question2'].apply(lambda x: pad_left_zeros(x,max_seq_length))

In [ ]:
# Split to dicts
X_train_dict = {'left': X_train_df['question1'], 'right': X_train_df['question2']}
X_valid_dict = {'left': X_valid_df['question1'], 'right': X_valid_df['question2']}
X_test_dict = {'left': X_test_df['question1'], 'right': X_test_df['question2']}

In [ ]:
# This layer can take as input a matrix
# and will return a vector of size 64
# shared_lstm = LSTM(64)

In [ ]:
# import tensorflow as tf;
# import keras
# print(keras.__version__)
# print(tf.__version__)
# python -c 'import keras; print(keras.__version__)'
# python3 -c 'import tensorflow as tf; print(tf.__version__)'  # for Python 3


In [ ]:
# left_output = shared_lstm(encoded_left)
# right_output = shared_lstm(encoded_right)


In [ ]:
# left_output.shape

In [ ]:
# We can then concatenate the two vectors:
# merged_vector = keras.layers.concatenate([left_output,right_output], axis=-1)

In [ ]:
# And add a logistic regression on top
# predictions = Dense(2, activation='softmax')(merged_vector)

In [ ]:
X_train_dict['left'] = np.concatenate(X_train_dict['left']).reshape(X_train_dict['left'].shape[0],max_seq_length)

In [ ]:
X_train_dict['left'].shape

In [ ]:
X_train_dict['right'] = np.concatenate(X_train_dict['right']).reshape(X_train_dict['right'].shape[0],max_seq_length)

In [ ]:
X_train_dict['right'].shape

In [ ]:
X_valid_dict['left'] = np.concatenate(X_valid_dict['left']).reshape(X_valid_dict['left'].shape[0],max_seq_length)

In [ ]:
X_valid_dict['right'] = np.concatenate(X_valid_dict['right']).reshape(X_valid_dict['right'].shape[0],max_seq_length)

In [ ]:
X_train_dict['right'][0]

In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)
#Input None,max_seq_length,1

left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embed_matrix),
                            embed_length,
                            weights=[embed_matrix],
                            input_length=max_seq_length,
                            trainable=False)

#Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# encoded_left.shape

# This layer can take as input a matrix
# and will return a vector of size 64
shared_lstm = LSTM(64)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)


# left_output.shape

# We can then concatenate the two vectors:
merged_vector = keras.layers.concatenate([left_output,right_output], axis=-1)

predictions = Dense(2, activation='softmax')(merged_vector)



model = Model(inputs=[left_input, right_input], outputs=predictions)

model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

n_epoch = 1


model.fit([X_train_dict['left'], X_train_dict['right']], Y_train, batch_size=128, epochs=n_epoch,
         validation_data=([X_valid_dict['left'], X_valid_dict['right']], Y_valid))



In [ ]:

def preprocess_model_input(input_questions,preprocess_text,words_to_index):

    final_new_words_list = []

    # i = 0
    for words_list in input_questions:
        new_words_list = []
      
        for word in preprocess_text(words_list):

            if words_to_index.get(word):
                new_words_list.append(words_to_index.get(word))
             
        final_new_words_list.append(np.array(pad_left_zeros(new_words_list,max_seq_length)))
        
         
    return [final_new_words_list[0].reshape(1,max_seq_length), final_new_words_list[1].reshape(1,max_seq_length)]

In [ ]:
def model_prediction(model,input_questions,preprocess_model_input,preprocess_text,words_to_index):
    y_prob = model.predict(preprocess_model_input(input_questions,preprocess_text,words_to_index),batch_size=1, verbose=0, steps=None)

    print ("The probabilities predicted by model for the question pair \n {} for classes [0,1] are \n {}".format(input_questions,y_prob))
    y_classes = y_prob.argmax(axis=-1)
    print ("The class that model predicted for the question pair \n {} is {}".format(input_questions,y_classes))
    return y_classes

In [ ]:
input_questions = ["What is the capital of India?", "What is the capital of India?"]
model_prediction(model,input_questions,preprocess_model_input,preprocess_text,words_to_index)


In [ ]:
input_questions = ["How can I be a good geologist?", "What should I do to be a great geologist?"]
model_prediction(model,input_questions,preprocess_model_input,preprocess_text,words_to_index)

In [ ]:
input_questions = ["What is the best travel website in spain?", "What is the best travel website?"]
model_prediction(model,input_questions,preprocess_model_input,preprocess_text,words_to_index)

In [ ]:

n_epoch = 4


model.fit([X_train_dict['left'], X_train_dict['right']], Y_train, batch_size=128, epochs=n_epoch,
         validation_data=([X_valid_dict['left'], X_valid_dict['right']], Y_valid))



In [ ]:

n_epoch = 6


model.fit([X_train_dict['left'], X_train_dict['right']], Y_train, batch_size=128, epochs=n_epoch,
         validation_data=([X_valid_dict['left'], X_valid_dict['right']], Y_valid))

In [ ]:
input_questions = ["What is the capital of India?", "What is the capital of India?"]
model_prediction(model,input_questions,preprocess_model_input,preprocess_text,words_to_index)

In [ ]:
input_questions = ["How can I be a good geologist?", "What should I do to be a great geologist?"]
model_prediction(model,input_questions,preprocess_model_input,preprocess_text,words_to_index)

In [ ]:
input_questions = ["What is the best travel website in spain?", "What is the best travel website?"]
model_prediction(model,input_questions,preprocess_model_input,preprocess_text,words_to_index)

In [ ]:

 
# # save the tokenizer and model
# with open("keras_tokenizer.pickle", "wb") as f:
#    pickle.dump(tokenizer, f)
model.save("../quora_keras_model_v2_tokenizer.hdf5")

In [ ]:
with open('words_to_index_v2.pickle', 'rb') as handle:
    words_to_index_1 = pickle.load(handle)

In [ ]:
with open('index_to_words_v2.pickle', 'rb') as handle:
    index_to_words_1 = pickle.load(handle)

In [ ]:
input_questions = ["How do I read and find my YouTube comments?", "How can I see all my Youtube comments?"]

In [ ]:
from keras.models import load_model

predict_model = load_model("../quora_keras_model_v2_tokenizer.hdf5")

In [ ]:
model_prediction(predict_model,input_questions,preprocess_model_input,preprocess_text,words_to_index)

# Increasing model layers to see if increase layers increase performance

In [ ]:
# expected input data shape: (batch_size, timesteps, data_dim)
#Input None,max_seq_length,1

# from keras.optimizers import Adam

left_input = Input(shape=(max_seq_length,), dtype='int32')
right_input = Input(shape=(max_seq_length,), dtype='int32')

embedding_layer = Embedding(len(embed_matrix),
                            embed_length,
                            weights=[embed_matrix],
                            input_length=max_seq_length,
                            trainable=False)

#Embedded version of the inputs
encoded_left = embedding_layer(left_input)
encoded_right = embedding_layer(right_input)

# encoded_left.shape

# This layer can take as input a matrix
# and will return a vector of size 64
shared_lstm = LSTM(64)

left_output = shared_lstm(encoded_left)
right_output = shared_lstm(encoded_right)


# left_output.shape

# We can then concatenate the two vectors:
merged_vector = keras.layers.concatenate([left_output,right_output], axis=-1)

merged_vector = Dense(64, activation='relu')(merged_vector)
merged_vector = Dense(64, activation='relu')(merged_vector)
merged_vector = Dense(64, activation='relu')(merged_vector)

predictions = Dense(2, activation='softmax')(merged_vector)



model_dense = Model(inputs=[left_input, right_input], outputs=predictions)

model_dense.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

n_epoch = 1


model_dense_hist = model_dense.fit([X_train_dict['left'], X_train_dict['right']], Y_train, batch_size=128, epochs=n_epoch,
         validation_data=([X_valid_dict['left'], X_valid_dict['right']], Y_valid))

In [ ]:

n_epoch = 4


model_dense_hist = model_dense.fit([X_train_dict['left'], X_train_dict['right']], Y_train, batch_size=128, epochs=n_epoch,
         validation_data=([X_valid_dict['left'], X_valid_dict['right']], Y_valid))

In [ ]:
n_epoch = 5


model_dense_hist = model_dense.fit([X_train_dict['left'], X_train_dict['right']], Y_train, batch_size=128, epochs=n_epoch,
         validation_data=([X_valid_dict['left'], X_valid_dict['right']], Y_valid))

In [ ]:
n_epoch = 1
model_dense_hist = model_dense.fit([X_train_dict['left'], X_train_dict['right']], Y_train, batch_size=128, epochs=n_epoch,
         validation_data=([X_valid_dict['left'], X_valid_dict['right']], Y_valid))

In [ ]:
n_epoch = 1
model_dense_hist = model_dense.fit([X_train_dict['left'], X_train_dict['right']], Y_train, batch_size=128, epochs=n_epoch,
         validation_data=([X_valid_dict['left'], X_valid_dict['right']], Y_valid))

In [ ]:
import matplotlib.pyplot as plt

# Plot accuracy
plt.plot(model_dense_hist.history['acc'])
plt.plot(model_dense_hist.history['val_acc'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot loss
plt.plot(model_dense_hist.history['loss'])
plt.plot(model_dense_hist.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
plt.show()

In [ ]:
print (model_dense_hist.history)

In [ ]:
input_questions = ["How can I be a good geologist?", "What should I do to be a great geologist?"]
model_prediction(model_dense,input_questions,preprocess_model_input,preprocess_text,words_to_index)

In [ ]:
input_questions = ["What is the best travel website in spain?", "What is the best travel website?"]
model_prediction(model_dense,input_questions,preprocess_model_input,preprocess_text,words_to_index)

In [ ]:
X_test_df.head(5)

In [ ]:
X_train_df.head(5)

In [ ]:
# re.sub(r"[^A-Za-z0-9,!.\/'+-=]", " ", 'why am i mentally very lonely? how can i solve')

In [ ]:
# re.sub(r"\'s", " ", 'what\'s')

In [ ]:
# words_list = []
# # train_subset_df = train_df['question1'][0:10]
# train_subset_df['question1'] = pd.DataFrame(data=train_df['question1'][0:10], columns=['question1'])
# train_subset_df['question2'] = pd.DataFrame(data=train_df['question2'][0:10], columns=['question2'])

In [ ]:
# for sentence in train_subset_df['question1']:
#     for word in sentence:
#         words_list.append(word)
# print (len(set(words_list)))

## Feature based on how many words are common in question 1 and question 2

In [ ]:
# u.termfrequency(['What is the step by step guide to invest in share market in india?'], ['What is the step by step guide to invest in share market?'])

In [ ]:
def termfrequency(sentence1, sentence2):
   
    question_dict ={}
    sentence1_words = sentence1   
    sentence2_words = sentence2
    searchtermfreq = []
    i = 0
    
    for key in sentence1_words:
#         print (key)
        question_dict[key] = question_dict.get(key,0) + 1
    
    for key in set(sentence2_words):
        value =  question_dict.get(key,0)
        if value >= 1:
            value = 1
        searchtermfreq.append(value)
        
    
#     print (question_dict)
#     print (searchtermfreq)
#     print (sum(searchtermfreq))
    return sum(searchtermfreq)

termfrequency(['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india?'], ['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market?'])
    
X_train_df['common_term_freq'] = X_train_df.apply(lambda x: termfrequency(x['question1'],x['question2']), axis=1 )

X_train_df.head(10)

X_valid_df['common_term_freq'] = X_valid_df.apply(lambda x: termfrequency(x['question1'],x['question2']), axis=1 )

X_valid_df.head(10)

# Total words frequency

In [ ]:
Total words frequency

def total_words_freq(sentence):
    return len(sentence)

total_words_freq(['what', 'is', 'the', 'step', 'by', 'step', 'guide', 'to', 'invest', 'in', 'share', 'market', 'in', 'india?'])

X_train_df['question1_words_freq'] = X_train_df['question1'].map(lambda x: total_words_freq(x))
X_train_df['question2_words_freq'] = X_train_df['question2'].map(lambda x: total_words_freq(x))

X_train_df.head(5)

X_valid_df['question1_words_freq'] = X_valid_df['question1'].map(lambda x: total_words_freq(x))
X_valid_df['question2_words_freq'] = X_valid_df['question2'].map(lambda x: total_words_freq(x))

X_valid_df.head(5)
X_train_model_input = X_train_df.drop(['question1','question2'],axis =1)
X_valid_model_input = X_valid_df.drop(['question1','question2'],axis =1)
X_train_model_input.head(3)



# Baseline Model

In [ ]:

# Train a model# Train  
import time

def train_classifier(clf, X_train, y_train):
    print ("Training {}...".format(clf.__class__.__name__))
    start = time.time()
    clf.fit(X_train, y_train)
    end = time.time()
    print ("Done!\nTraining time (secs): {:.3f}".format(end - start))
    

from sklearn.linear_model import LogisticRegression

clf =  LogisticRegression()

train_classifier(clf, X_train_model_input, y_train.values.ravel())

from sklearn.metrics import log_loss
from sklearn.metrics import confusion_matrix

def predict_labels(clf, X_train, y_train):
    print ("Predicting labels using {}...".format(clf.__class__.__name__))
    start = time.time()
    y_pred = clf.predict(X_train)
    end = time.time()
    print ("Done!\nPrediction time (secs): {:.3f}".format(end - start))
    return log_loss(y_train, y_pred, eps=1e-15), confusion_matrix(y_train, y_pred)

train_metrics = predict_labels(clf, X_train_model_input, y_train.values.ravel())

print 
print ("Log loss for training set: {}".format(train_metrics[0]))

print ("Confusion matrix for training set: {}".format(train_metrics[1]))

# Predict on test data
print ("Log loss for validation set: {}".format(predict_labels(clf, X_valid_model_input, y_valid.values.ravel())[0]))

print ("Confusion matrix for validation set: {}".format(predict_labels(clf, X_valid_model_input, y_valid.values.ravel())[1]))



# Text Preprocessing

In [ ]:
words_list = create_vocabulary([],X_train_df,'question1')
print ("Lenght of words in X_train_df question 1 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,X_train_df,'question2')
print ("Lenght of words after adding X_train_df question 2 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,X_valid_df,'question1')
print ("Lenght of words after adding X_valid_df question 1 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,X_valid_df,'question2')
print ("Lenght of words after adding X_valid_df question 2 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,X_test_df,'question1')
print ("Lenght of words after adding X_test_df question 1 {}".format(len(words_list)))
words_list = create_vocabulary(words_list,X_test_df,'question2')
print ("Lenght of words after adding X_test_df question 2 {}".format(len(words_list)))



In [ ]:

def preprocess_text(list_words):
     list_words_processed = []
     for text in list_words:
         text = re.sub(r"\?", '', text)
         text = re.sub(r"i'm", "i am ", text)
#          print (text)
         list_words_processed.append(str(text))
#          print (list_words_processed)
     return list_words_processed

In [ ]:
preprocess_text(['India?'])

In [ ]:
preprocess_text(['how',
 'do',
 'the',
 'holy',
 'scriptures',
 'of',
 'hinduism',
 'compare',
 'and',
 'contrast',
 'to',
 'those',
 'of',
 'taoism?'])

In [ ]:
X_train_df['question1'][3]

In [ ]:
preprocess_text(X_train_df['question1'][3])

In [ ]:
X_train_p_df = pd.DataFrame()
X_valid_p_df = pd.DataFrame()
X_test_p_df = pd.DataFrame()

In [ ]:
X_train_p_df['question1'] = X_train_df['question1'].apply(lambda x:preprocess_text(x))
X_train_p_df['question2'] = X_train_df['question2'].apply(lambda x:preprocess_text(x))
X_valid_p_df['question1'] = X_valid_df['question1'].apply(lambda x:preprocess_text(x))
X_valid_p_df['question2'] = X_valid_df['question2'].apply(lambda x:preprocess_text(x))
X_test_p_df['question1'] = X_test_df['question1'].apply(lambda x:preprocess_text(x))
X_test_p_df['question2'] = X_test_df['question2'].apply(lambda x:preprocess_text(x))

In [ ]:
X_train_p_df['question1'][2]

In [ ]:
proc_words_list = create_vocabulary([],X_train_p_df,'question1')
print ("Lenght of words in X_train_df question 1 {}".format(len(words_list)))
proc_words_list = create_vocabulary(words_list,X_train_p_df,'question2')
print ("Lenght of words after adding X_train_df question 2 {}".format(len(words_list)))
proc_words_list = create_vocabulary(words_list,X_valid_p_df,'question1')
print ("Lenght of words after adding X_valid_df question 1 {}".format(len(words_list)))
proc_words_list = create_vocabulary(words_list,X_valid_p_df,'question2')
print ("Lenght of words after adding X_valid_df question 2 {}".format(len(words_list)))
proc_words_list = create_vocabulary(words_list,X_test_p_df,'question1')
print ("Lenght of words after adding X_test_df question 1 {}".format(len(words_list)))
proc_words_list = create_vocabulary(words_list,X_test_p_df,'question2')
print ("Lenght of words after adding X_test_df question 2 {}".format(len(words_list)))



In [ ]:
proc_words_freq = collections.Counter(proc_words_list)

In [ ]:
proc_words_freq_10000 = proc_words_freq.most_common(10000)

In [ ]:
proc_word_in_word2vec = []
proc_word_notin_word2vec = []

for word in proc_words_freq.most_common(10000):
    if word[0] in model.vocab:
        proc_word_in_word2vec.append(word[0])
    else:
        proc_word_notin_word2vec.append(word[0])

In [ ]:
print (len(proc_word_in_word2vec))
print (len(proc_word_notin_word2vec))
print (proc_word_notin_word2vec[0:100])

In [ ]:
print (proc_word_in_word2vec[0:100])

# Pipeline

# Keras

## Create Vocabulary

In [ ]:
#